In [1]:
import os
import sys
from pathlib import Path

from loguru import logger
from reload_recursive import reload_recursive
from typing import Callable

from nipype.interfaces import fsl

import monai_training
import mri_data
import sys

In [10]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data import file_manager as fm
from mri_data.file_manager import Scan, DataSet
from mri_data import utils
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
drive_root = fm.get_drive_root()
projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
inference_root = drive_root / "3Tpioneer_bids_predictions"
dataroot = drive_root / "3Tpioneer_bids"

workspace_root = "/home/srs-9/Projects/ms_mri/itksnap_workspaces"
dataset_name = "choroid_pineal_pituitary_labels"

save_dir = os.path.join(workspace_root, dataset_name)

In [5]:
dataset_proc = DataSetProcesser.new_dataset(dataroot, fm.scan_3Tpioneer_bids, filters=[fm.filter_first_ses])
dataset = dataset_proc.dataset
dataset.sort()
inference_dataset = DataSet.dataset_like(dataset)
inference_dataset.dataroot = inference_root

In [6]:
image_names = ["flair.nii.gz", "t1.nii.gz"]
label_names = []
inference_labels = ["flair.t1_pineal1_pred.nii.gz", "flair.t1_choroid_pineal_pituitary2_pred.nii.gz"]

In [11]:
inds = slice(0,25)
for scan, inf_scan in zip(dataset[inds], inference_dataset[inds]):
    images = [scan.root / im for im in image_names if (scan.root / im).is_file()]
    man_labels = [scan.root / lab for lab in label_names if (scan.root / lab).is_file()]
    inferences = [inf_scan.root / lab for lab in inference_labels if (inf_scan.root / lab).is_file()]
    labels = man_labels + inferences
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=True)
    print(cmd)

itksnap -g H:/3Tpioneer_bids/sub-ms1001/ses-20170215/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1001/ses-20170215/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/flair.t1_pineal1_pred.nii.gz H:/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/flair.t1_choroid_pineal_pituitary2_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1002/ses-20200521/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1002/ses-20200521/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521/flair.t1_pineal1_pred.nii.gz H:/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521/flair.t1_choroid_pineal_pituitary2_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1003/ses-20170329/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1003/ses-20170329/t1.nii.gz -s H:/3Tpioneer_bids_predictions/sub-ms1003/ses-20170329/flair.t1_pineal1_pred.nii.gz H:/3Tpioneer_bids_predictions/sub-ms1003/ses-20170329/flair.t1_choroid_pineal_pituitary2_pred.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1004/ses-20190906/flair.nii.gz -o H:/3Tpi

In [18]:
for scan, inf_scan in zip(dataset[inds], inference_dataset[inds]):
    print(inf_scan)

Scan(subid='1001', sesid='20170215', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids_predictions'), _root=PosixPath('/media/smbshare/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215'), image=None, label=None, cond=None, id=20190385215)
Scan(subid='1002', sesid='20200521', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids_predictions'), _root=PosixPath('/media/smbshare/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521'), image=None, label=None, cond=None, id=20240922042)
Scan(subid='1003', sesid='20170329', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids_predictions'), _root=PosixPath('/media/smbshare/3Tpioneer_bids_predictions/sub-ms1003/ses-20170329'), image=None, label=None, cond=None, id=20230839987)
Scan(subid='1004', sesid='20190906', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids_predictions'), _root=PosixPath('/media/smbshare/3Tpioneer_bids_predictions/sub-ms1004/ses-20190906'), image=None, label=None, cond=None, id=20271669624)
Scan(subid='1005', sesid='202007